In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional as F

import wandb
import os

# Loading Data:
---

In [2]:
file_path = '../data/test_data.csv'
data = pd.read_csv(file_path)
file_path = '../data/name_smiles_embedding_file.csv'
name_smiles_embedding_df = pd.read_csv(file_path)

In [3]:
data.head()

,Unnamed: 0,index,p_184,p_185,p_186,p_187,p_188,p_189,p_190,p_191,...,n_1021,Label,DEB,DEM,DMMP,DPM,DtBP,JP8,MES,TEPO
0,298520,1364474,28.0,27.0,26.0,24.0,22.0,22.0,20.0,20.0,...,6.0,DMMP,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,77906,477962,96.0,99.0,103.0,105.0,108.0,111.0,113.0,117.0,...,-14.0,DEM,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,149391,843445,87.0,87.0,86.0,86.0,86.0,87.0,89.0,92.0,...,-14.0,TEPO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,262066,1262888,97.0,100.0,104.0,106.0,111.0,115.0,118.0,124.0,...,-3.0,DMMP,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,279662,1321891,122.0,128.0,133.0,140.0,146.0,153.0,159.0,169.0,...,-4.0,TEPO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [20]:
data.columns[-8:]

Index(['DEB', 'DEM', 'DMMP', 'DPM', 'DtBP', 'JP8', 'MES', 'TEPO'], dtype='object')

In [5]:
name_smiles_embedding_df.head()

,Unnamed: 0,Name,SMILES,embedding
0,BKG,background,NaN,NaN
1,DEM,Diethyl Malonate,CCOC(=O)CC(=O)OCC,"[0.3809721, 0.0005454041, 0.25539744, -0.24272..."
2,DEB,"1,2,3,4-Diepoxybutane",C1C(O1)C2CO2,"[0.06318794, 0.009022224, 0.42160064, 0.195722..."
3,MES,2-(N-morpholino)ethanesulfonic acid,C1COCCN1CCS(=O)(=O)O,"[-0.32520828, 0.009838344, -0.15108332, 0.2845..."
4,DMMP,Dimethyl methylphosphonate,COP(=O)(C)OC,"[0.12106811, 0.00294244, -0.14450458, 0.072665..."


In [6]:
# set the df index to be the chemical abbreviations in col 'Unnamed: 0'
name_smiles_embedding_df.set_index('Unnamed: 0', inplace=True)
name_smiles_embedding_df.head()

,Name,SMILES,embedding
Unnamed: 0,,,
BKG,background,NaN,NaN
DEM,Diethyl Malonate,CCOC(=O)CC(=O)OCC,"[0.3809721, 0.0005454041, 0.25539744, -0.24272..."
DEB,"1,2,3,4-Diepoxybutane",C1C(O1)C2CO2,"[0.06318794, 0.009022224, 0.42160064, 0.195722..."
MES,2-(N-morpholino)ethanesulfonic acid,C1COCCN1CCS(=O)(=O)O,"[-0.32520828, 0.009838344, -0.15108332, 0.2845..."
DMMP,Dimethyl methylphosphonate,COP(=O)(C)OC,"[0.12106811, 0.00294244, -0.14450458, 0.072665..."


In [7]:
name_smiles_embedding_df['SMILES']['DEM']

# name_smiles_embedding_df.head()

'CCOC(=O)CC(=O)OCC'

# Definitions:
---

In [17]:
class Encoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.encoder = nn.Sequential(
      nn.Linear(1676,1000),
      nn.LeakyReLU(inplace=True),
      nn.Linear(1000,512),
      # nn.LeakyReLU(inplace=True),
      # nn.Linear(733,701),
      # nn.LeakyReLU(inplace=True),
      # nn.Linear(701,669),
      # nn.LeakyReLU(inplace=True),
      # nn.Linear(669, 637),
      # nn.LeakyReLU(inplace=True),
      # nn.Linear(637, 605),
      # nn.LeakyReLU(inplace=True),
      # nn.Linear(605, 573),
      # nn.LeakyReLU(inplace=True),
      # nn.Linear(573, 541),
      # nn.LeakyReLU(inplace=True),
      # nn.Linear(541, 512),
    )

  def forward(self, x):
    x = self.encoder(x)
    return x

In [9]:
def run_with_wandb(config, **kwargs):
    config.update(kwargs)

    wandb.init(entity=config['wandb_entity'],
               project=config['wandb_project'],
               config=config)

    # Set the number of threads
    torch.set_num_threads(config['threads'])

    # Find out is there is a GPU available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    if not config['gpu']:
        device = torch.device('cpu')
    print(f'Using device: {device}')

# Creating Dataloader Objects
---

In [10]:
name_smiles_embedding_df.head()

,Name,SMILES,embedding
Unnamed: 0,,,
BKG,background,NaN,NaN
DEM,Diethyl Malonate,CCOC(=O)CC(=O)OCC,"[0.3809721, 0.0005454041, 0.25539744, -0.24272..."
DEB,"1,2,3,4-Diepoxybutane",C1C(O1)C2CO2,"[0.06318794, 0.009022224, 0.42160064, 0.195722..."
MES,2-(N-morpholino)ethanesulfonic acid,C1COCCN1CCS(=O)(=O)O,"[-0.32520828, 0.009838344, -0.15108332, 0.2845..."
DMMP,Dimethyl methylphosphonate,COP(=O)(C)OC,"[0.12106811, 0.00294244, -0.14450458, 0.072665..."


Currently embeddings are strings instead of a list of floats. Would be good to fix that in preprocessing.

In [11]:
embedding_floats = []
for chem_name in name_smiles_embedding_df.index:
    if chem_name == 'BKG':
        embedding_floats.append(None)
    else:
        embedding_float = name_smiles_embedding_df['embedding'][chem_name].split('[')[1]
        embedding_float = embedding_float.split(']')[0]
        embedding_float = [np.float32(num) for num in embedding_float.split(',')]
        embedding_floats.append(embedding_float)

name_smiles_embedding_df['Embedding Floats'] = embedding_floats

In [12]:
# drop first two cols ('Unnamed:0' and 'index') and last 9 cols ('Label' and OneHot encodings) to get just spectra
train_spectra = data.iloc[:,2:-9]
train_chem_encodings = data.iloc[:,-8:]

In [13]:
train_chem_labels = list(data['Label'])
train_embeddings_tensor = torch.Tensor([name_smiles_embedding_df['Embedding Floats'][chem_name] for chem_name in train_chem_labels])
train_spectra_tensor = torch.Tensor(train_spectra.values)
train_chem_encodings_tensor = torch.Tensor(train_chem_encodings.values)

batch_size = 32
train_dataset = DataLoader(TensorDataset(train_spectra_tensor, train_chem_encodings_tensor, train_embeddings_tensor), batch_size=batch_size, shuffle=True)

# Train Encoder:
---

In [15]:
# os.environ['WANDB_API_KEY'] = '9729ad7b1f3a60f1072cdf7af979b737955733d4'
config = {
    'wandb_entity': 'catemerfeld',
    'wandb_project': 'ims_encoder_decoder',
    'gpu':True,
    'threads':1,
}

os.environ['WANDB_NOTEBOOK_NAME'] = '/home/cmdunham/ChemicalDataGeneration/models/ims_encoder.ipynb'

In [24]:
# These two should be the same
sorted_chem_names = list(data.columns[-8:])
# sorted_chem_names = ['DEB', 'DEM', 'DMMP', 'DPM', 'DtBP', 'JP8', 'MES', 'TEPO']

epochs = 10
learning_rate = .0001
encoder = Encoder()

encoder_optimizer = torch.optim.AdamW(encoder.parameters(), lr = learning_rate)
encoder_criterion = nn.MSELoss()

predicted_embeddings = []
output_chem_names = []

wandb_kwargs = {
    'learning_rate': learning_rate,
    'epochs': epochs,
    'batch_size': batch_size,
    'model_architecture': 'encoder',
    'optimizer':'AdamW',
    'loss': 'MSELoss'
}

run_with_wandb(config, **wandb_kwargs)

for epoch in range(epochs):
  # accumulate epoch loss
  epoch_training_loss = 0

  for batch, name_encodings, true_embeddings in train_dataset:
    batch_predicted_embeddings = encoder(batch)

    loss = encoder_criterion(batch_predicted_embeddings, true_embeddings)
    epoch_training_loss += loss

    # back propogation
    encoder_optimizer.zero_grad()
    loss.backward()
    encoder_optimizer.step()

    # at last epoch store output embeddings and corresponding labels to output list
    if (epoch + 1) == epochs:
      chem_names = [sorted_chem_names[list(encoding).index(1)] for encoding in name_encodings]
      for emb, chem_name in zip(batch_predicted_embeddings, chem_names):
        predicted_embeddings.append(emb)
        output_chem_names.append(chem_name)

  # encoder.eval() # Set model to evaluation mode
  # epoch_test_loss = 0
  # with torch.no_grad():
  #   for batch, name_encodings, true_embeddings in val_dataset:
  #     batch_predicted_embeddings = encoder(batch)

  #     loss = encoder_criterion(batch_predicted_embeddings, true_embeddings)
  #     epoch_test_loss += loss

  # # Set models back to training mode
  # encoder.train()

  average_loss = epoch_training_loss/len(train_dataset)
  # test_average_loss = epoch_test_loss/len(val_dataset)
  wandb.log({"Encoder Training Loss": average_loss})#, "Encoder Test Loss": test_average_loss})

  # wandb.log({"Encoder Test Loss": test_average_loss})

  if (epoch + 1) % 10 == 0:
    print('Epoch[{}/{}], loss: {}'.format(epoch+1, epochs, average_loss))
    # print('Test loss: {:.4f}'.format(test_average_loss))
    print('-------------------------------------------')

wandb.finish()